## Install Cohere

In [ ]:
!pip install cohere

## Connect to Cohere API

In [169]:
import cohere
api_key = 'wMOHw4sYHDcg2Pn0ky7hbA8DFvjj6rUGJQ0Vcm6l'
co = cohere.Client(api_key)

## Import dataset and show a few rows
This dataset originates from this Kaggle competition (https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/overview) and was cleaned to include only labels of "toxic" and "benign" in this GitHub repository (https://github.com/ericrisco/twitch-chat-toxicity?tab=readme-ov-file)


In [170]:
import pandas as pd

twitch_data = pd.read_csv("drive/MyDrive/UofT/Year 5 Semester 2/UofTHacks/train_clean.csv")
twitch_data.head()

,text,label
0,Explanation\nWhy the edits made under my usern...,benign
1,D'aww! He matches this background colour I'm s...,benign
2,"Hey man, I'm really not trying to edit war. It...",benign
3,"""\nMore\nI can't make any real suggestions on ...",benign
4,"You, sir, are my hero. Any chance you remember...",benign


## Remove newline characters from the text and clear whitespace at beginning and end of text

In [171]:
twitch_data['text'] = twitch_data['text'].apply(lambda x : x.replace("\n", " "))
twitch_data['text'] = twitch_data['text'].apply(lambda x : x.strip())
twitch_data.head()

,text,label
0,Explanation Why the edits made under my userna...,benign
1,D'aww! He matches this background colour I'm s...,benign
2,"Hey man, I'm really not trying to edit war. It...",benign
3,""" More I can't make any real suggestions on im...",benign
4,"You, sir, are my hero. Any chance you remember...",benign


## Randomly select 1250 benign messages and 1250 toxic ones
As the model can only accept 2500 text-label pairs as examples, we randomly choose 1250 messages from each category

In [172]:
import numpy as np

np.random.seed(1234)

benign_indexes = twitch_data.index[twitch_data['label'] == 'benign'].tolist()
selected_benign = np.random.choice(benign_indexes, 1250, replace=False)
benign_data = twitch_data.iloc[selected_benign]

toxic_indexes = twitch_data.index[twitch_data['label'] == 'toxic'].tolist()
selected_toxic = np.random.choice(toxic_indexes, 1250, replace=False)
toxic_data = twitch_data.iloc[selected_toxic]

## Create new dataset of selected benign messages and export to JSON

In [183]:
benign_data = benign_data.reset_index(drop=True)
benign_data.head()
# benign_data.to_json("benign_data.json")

,text,label
0,"Yes, Reversider is of course completely correc...",benign
1,"Try harder, the evidence is even on Wiki Fred ...",benign
2,""" I know you are writing these things in good...",benign
3,Arktos the troublemaker and wik vandal who doe...,benign
4,""", and welcome to Wikipedia! Thank you for you...",benign


## Create new dataset of selected toxic messages and export to JSON

In [184]:
toxic_data = toxic_data.reset_index(drop=True)
toxic_data.head()
# toxic_data.to_json("toxic_data.json")

,text,label
0,"No, it's not just you. Just more evidence that...",toxic
1,no abusing bots and stop being an idiot the pe...,toxic
2,You horrid little mincing poofter I've got m...,toxic
3,Quite so. Bullshit artists - like every other ...,toxic
4,"Enough vandalism, child. Your edits are neithe...",toxic


## Create the Example objects that will be passed into the classifier

In [175]:
from cohere.responses.classify import Example
examples = []
for i in range(len(benign_data)):
  examples.append(Example(benign_data.iloc[i]['text'], 'benign'))
for i in range(len(toxic_data)):
  examples.append(Example(toxic_data.iloc[i]['text'], 'toxic'))

## Test the classifier using the examples

In [176]:
response = co.classify(
  inputs=['you suck', 'can\'t wait to make more memories with you!'],
  model='embed-english-v2.0',
  examples=examples
)
for classification in response:
  print(classification.predictions)

['toxic']
['benign']
